##### <b>Install Library</b>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time

##### <b>SET50 Index Futures</b>

In [3]:
data = pd.read_csv('SET50_Index_Futures.csv')
data = data[(data['Symbol'] != 'S50M06') & (data['Symbol'] != 'S50H07') & (data['Symbol'] != 'S50M07') & (data['Symbol'] != 'S50U07')].reset_index(drop=True)
data = data.pivot(index='Date', columns='Symbol', values='SP').reset_index().drop_duplicates(subset='Date', keep='last')
data['Date'] = pd.to_datetime(data['Date'])
data = data[data['Date'] >= '2007-10-29'].reset_index(drop=True)
data

Symbol,Date,S50H08,S50H09,S50H10,S50H11,S50H12,S50H13,S50H14,S50H15,S50H16,...,S50Z14,S50Z15,S50Z16,S50Z17,S50Z18,S50Z19,S50Z20,S50Z21,S50Z22,S50Z23
0,2007-10-29,693.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-10-30,681.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-10-31,688.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-11-01,677.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-11-02,673.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918,2023-11-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,864.0
3919,2023-11-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,861.0
3920,2023-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,868.0
3921,2023-11-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,858.0


##### <b>Manipulate SET50 Index Futures</b>

In [4]:
def round_to_nearest_25(value):
    return 25 * round(value / 25)

def round_to_nearest_50(value):
    return 50 * round(value / 50)

def vlookup_call(row, file_prefix):
    strike_price = row['Strike Price']
    column_name = f'{file_prefix}C{strike_price}'
    atm_value = row[column_name]

    return atm_value

def vlookup_put(row, file_prefix):
    strike_price = row['Strike Price']
    column_name = f'{file_prefix}P{strike_price}'
    atm_value = row[column_name]

    return atm_value

column_names_25 = ['S50H14', 'S50H15', 'S50H16', 'S50H17', 'S50H18', 'S50H19', 'S50H20', 'S50H21', 'S50H22', 'S50H23', 
                   'S50M14', 'S50M15', 'S50M16', 'S50M17', 'S50M18', 'S50M19', 'S50M20', 'S50M21', 'S50M22', 'S50M23', 
                   'S50U13', 'S50U14', 'S50U15', 'S50U16', 'S50U17', 'S50U18', 'S50U19', 'S50U20', 'S50U21', 'S50U22', 'S50U23', 
                   'S50Z13', 'S50Z14', 'S50Z15', 'S50Z16', 'S50Z17', 'S50Z18', 'S50Z19', 'S50Z20', 'S50Z21', 'S50Z22', 'S50Z23']
    
column_names_50 = ['S50H08', 'S50H09', 'S50H10', 'S50H11', 'S50H12', 'S50H13',
                   'S50M08', 'S50M09', 'S50M10', 'S50M11', 'S50M12', 'S50M13',
                   'S50U06', 'S50U08', 'S50U09', 'S50U10', 'S50U11', 'S50U12',
                   'S50Z06', 'S50Z07', 'S50Z08', 'S50Z09', 'S50Z10', 'S50Z11', 'S50Z12']

# Dictionary to store DataFrames
dfs_25 = {}
dfs_50 = {}

# Create DataFrames for 'column_names_25'
for column in column_names_25:
    selected_columns = ['Date', column]
    new_df_25 = data[selected_columns].dropna().reset_index(drop=True)
    
    # Calculate Strike_Price column with the round_to_nearest_25 function
    new_df_25['Strike Price'] = new_df_25[column].apply(round_to_nearest_25)
    
    # Store the DataFrame in the dictionary
    dfs_25[column] = new_df_25

# Create DataFrames for 'column_names_50'
for column in column_names_50:
    selected_columns = ['Date', column]
    new_df_50 = data[selected_columns].dropna().reset_index(drop=True)
    
    # Calculate Strike_Price column with the round_to_nearest_50 function
    new_df_50['Strike Price'] = new_df_50[column].apply(round_to_nearest_50)
    
    # Store the DataFrame in the dictionary
    dfs_50[column] = new_df_50

# Interval 25
S50H14 = dfs_25['S50H14']; S50H15 = dfs_25['S50H15']; S50H16 = dfs_25['S50H16']; S50H17 = dfs_25['S50H17']
S50H18 = dfs_25['S50H18']; S50H19 = dfs_25['S50H19']; S50H20 = dfs_25['S50H20']; S50H21 = dfs_25['S50H21']
S50H22 = dfs_25['S50H22']; S50H23 = dfs_25['S50H23']

S50M14 = dfs_25['S50M14']; S50M15 = dfs_25['S50M15']; S50M16 = dfs_25['S50M16']; S50M17 = dfs_25['S50M17']
S50M18 = dfs_25['S50M18']; S50M19 = dfs_25['S50M19']; S50M20 = dfs_25['S50M20']; S50M21 = dfs_25['S50M21']
S50M22 = dfs_25['S50M22']; S50M23 = dfs_25['S50M23']

S50U13 = dfs_25['S50U13']; S50U14 = dfs_25['S50U14']; S50U15 = dfs_25['S50U15']; S50U16 = dfs_25['S50U16']
S50U17 = dfs_25['S50U17']; S50U18 = dfs_25['S50U18']; S50U19 = dfs_25['S50U19']; S50U20 = dfs_25['S50U20']
S50U21 = dfs_25['S50U21']; S50U22 = dfs_25['S50U22']; S50U23 = dfs_25['S50U23']

S50Z13 = dfs_25['S50Z13']; S50Z14 = dfs_25['S50Z14']; S50Z15 = dfs_25['S50Z15']; S50Z16 = dfs_25['S50Z16']
S50Z17 = dfs_25['S50Z17']; S50Z18 = dfs_25['S50Z18']; S50Z19 = dfs_25['S50Z19']; S50Z20 = dfs_25['S50Z20']
S50Z21 = dfs_25['S50Z21']; S50Z22 = dfs_25['S50Z22']; S50Z23 = dfs_25['S50Z23']

# Interval 50
S50H08 = dfs_50['S50H08']; S50H09 = dfs_50['S50H09']; S50H10 = dfs_50['S50H10']; S50H11 = dfs_50['S50H11']
S50H12 = dfs_50['S50H12']; S50H13 = dfs_50['S50H13']

S50M08 = dfs_50['S50M08']; S50M09 = dfs_50['S50M09']; S50M10 = dfs_50['S50M10']; S50M11 = dfs_50['S50M11']
S50M12 = dfs_50['S50M12']; S50M13 = dfs_50['S50M13']

S50U08 = dfs_50['S50U08']; S50U09 = dfs_50['S50U09']; S50U10 = dfs_50['S50U10']; S50U11 = dfs_50['S50U11']
S50U12 = dfs_50['S50U12']

S50Z07 = dfs_50['S50Z07']; S50Z08 = dfs_50['S50Z08']; S50Z09 = dfs_50['S50Z09']; S50Z10 = dfs_50['S50Z10']
S50Z11 = dfs_50['S50Z11']; S50Z12 = dfs_50['S50Z12']

##### <b>Combined SET50 Index Futures and SET50 Index Options</b>

###### <b>Create Functions</b>

In [5]:
def process_options_data(futures_symbol):
    Call_Options_data = pd.read_csv(f'{futures_symbol}_Call_options_data.csv')
    Call_Options = Call_Options_data[['Date', 'Symbol', 'SP']]
    Call_Options = Call_Options.sort_values(by='Date').reset_index(drop=True)

    Put_Options_data = pd.read_csv(f'{futures_symbol}_Put_options_data.csv')
    Put_Options = Put_Options_data[['Date', 'Symbol', 'SP']]
    Put_Options = Put_Options.sort_values(by='Date').reset_index(drop=True)

    # Select options first day trade
    first_day = Call_Options['Date'].min()
    Futures['Date'] = pd.to_datetime(Futures['Date'])
    mask = Futures['Date'] >= pd.to_datetime(first_day)
    Futures_subset = Futures[mask].reset_index(drop=True)
    Futures_subset['Date'] = pd.to_datetime(Futures_subset['Date'])
    Futures_subset = Futures_subset.sort_values(by='Date').reset_index(drop=True)

    call_options_unique_symbols = Call_Options['Symbol'].unique()
    symbol_dataframes = {}

    for symbol in call_options_unique_symbols:
        symbol_df = pd.DataFrame({
            'Date': Call_Options[Call_Options['Symbol'] == symbol]['Date'],
            symbol: Call_Options[Call_Options['Symbol'] == symbol]['SP']
        })
        symbol_dataframes[symbol] = symbol_df

    merged_df = symbol_dataframes[call_options_unique_symbols[0]]
    for symbol in call_options_unique_symbols[1:]:
        merged_df = pd.merge(merged_df, symbol_dataframes[symbol], on='Date', how='outer')

    Call_Options_Clean = merged_df.drop_duplicates(subset=['Date']).sort_values(by='Date').reset_index(drop=True).fillna(0)
    Call_Options_Clean['Date'] = pd.to_datetime(Call_Options_Clean['Date'])

    put_options_unique_symbols = Put_Options['Symbol'].unique()
    symbol_dataframes = {}

    for symbol in put_options_unique_symbols:
        symbol_df = pd.DataFrame({
            'Date': Put_Options[Put_Options['Symbol'] == symbol]['Date'],
            symbol: Put_Options[Put_Options['Symbol'] == symbol]['SP']
        })
        symbol_dataframes[symbol] = symbol_df

    merged_df = symbol_dataframes[put_options_unique_symbols[0]]
    for symbol in put_options_unique_symbols[1:]:
        merged_df = pd.merge(merged_df, symbol_dataframes[symbol], on='Date', how='outer')

    Put_Options_Clean = merged_df.drop_duplicates(subset=['Date']).sort_values(by='Date').reset_index(drop=True).fillna(0)
    Put_Options_Clean['Date'] = pd.to_datetime(Put_Options_Clean['Date'])

    Data = Futures_subset.merge(Call_Options_Clean, on='Date', how='outer').fillna(0)
    Data = Data.merge(Put_Options_Clean, on='Date', how='outer').fillna(0)
    Data['Date'] = pd.to_datetime(Data['Date'])
    Data = Data.sort_values(by='Date').reset_index(drop=True)
    Data['Strike Price'] = Data['Strike Price'].astype(int)
    Data['Call ATM'] = Data.apply(vlookup_call, axis=1, args=(futures_symbol,))
    Data['Put ATM'] = Data.apply(vlookup_put, axis=1, args=(futures_symbol,))
    Data = Data[['Date', 'Strike Price', 'Call ATM', 'Put ATM']]
    Data['Sum Premium'] = (Data['Call ATM'] + Data['Put ATM'])
    Data['Normalized Premium'] = Data['Strike Price'] / Data['Sum Premium']

    return Data

###### <b>S50U13 to S50Z23</b>

In [ ]:
# 2013
Futures = S50U13
Futures_symbol = 'S50U13'
S50U13 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z13
Futures_symbol = 'S50Z13'
S50Z13 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

# Options_2013 = pd.concat([S50H13, S50M13, S50U13, S50Z13], axis=1)
Options_2013 = pd.concat([S50U13, S50Z13], axis=1)
Options_2013

In [ ]:
# 2014
Futures = S50H14
Futures_symbol = 'S50H14'
S50H14 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M14
Futures_symbol = 'S50M14'
S50M14 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U14
Futures_symbol = 'S50U14'
S50U14 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z14
Futures_symbol = 'S50Z14'
S50Z14 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2014 = pd.concat([S50H14, S50M14, S50U14, S50Z14], axis=1)
Options_2014

In [ ]:
# 2015
Futures = S50H15
Futures_symbol = 'S50H15'
S50H15 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M15
Futures_symbol = 'S50M15'
S50M15 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U15
Futures_symbol = 'S50U15'
S50U15 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z15
Futures_symbol = 'S50Z15'
S50Z15 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2015 = pd.concat([S50H15, S50M15, S50U15, S50Z15], axis=1)
Options_2015

In [ ]:
# 2016
Futures = S50H16
Futures_symbol = 'S50H16'
S50H16 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M16
Futures_symbol = 'S50M16'
S50M16 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U16
Futures_symbol = 'S50U16'
S50U16 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z16
Futures_symbol = 'S50Z16'
S50Z16 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2016 = pd.concat([S50H16, S50M16, S50U16, S50Z16], axis=1)
Options_2016

In [ ]:
# 2017
Futures = S50H17
Futures_symbol = 'S50H17'
S50H17 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M17
Futures_symbol = 'S50M17'
S50M17 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U17
Futures_symbol = 'S50U17'
S50U17 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z17
Futures_symbol = 'S50Z17'
S50Z17 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2017 = pd.concat([S50H17, S50M17, S50U17, S50Z17], axis=1)
Options_2017

In [ ]:
# 2018
Futures = S50H18
Futures_symbol = 'S50H18'
S50H18 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M18
Futures_symbol = 'S50M18'
S50M18 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U18
Futures_symbol = 'S50U18'
S50U18 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z18
Futures_symbol = 'S50Z18'
S50Z18 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2018 = pd.concat([S50H18, S50M18, S50U18, S50Z18], axis=1)
Options_2018

In [ ]:
# 2019
Futures = S50H19
Futures_symbol = 'S50H19'
S50H19 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M19
Futures_symbol = 'S50M19'
S50M19 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U19
Futures_symbol = 'S50U19'
S50U19 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z19
Futures_symbol = 'S50Z19'
S50Z19 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2019 = pd.concat([S50H19, S50M19, S50U19, S50Z19], axis=1)
Options_2019

In [ ]:
# 2020
Futures = S50H20
Futures_symbol = 'S50H20'
S50H20 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M20
Futures_symbol = 'S50M20'
S50M20 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U20
Futures_symbol = 'S50U20'
S50U20 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z20
Futures_symbol = 'S50Z20'
S50Z20 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2020 = pd.concat([S50H20, S50M20, S50U20, S50Z20], axis=1)
Options_2020

In [ ]:
# 2021
Futures = S50H21
Futures_symbol = 'S50H21'
S50H21 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M21
Futures_symbol = 'S50M21'
S50M21 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U21
Futures_symbol = 'S50U21'
S50U21 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z21
Futures_symbol = 'S50Z21'
S50Z21 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2021 = pd.concat([S50H21, S50M21, S50U21, S50Z21], axis=1)
Options_2021

In [ ]:
# 2022
Futures = S50H22
Futures_symbol = 'S50H22'
S50H22 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M22
Futures_symbol = 'S50M22'
S50M22 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U22
Futures_symbol = 'S50U22'
S50U22 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z22
Futures_symbol = 'S50Z22'
S50Z22 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2022 = pd.concat([S50H22, S50M22, S50U22, S50Z22], axis=1)
Options_2022

In [ ]:
# 2023
Futures = S50H23
Futures_symbol = 'S50H23'
S50H23 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M23
Futures_symbol = 'S50M23'
S50M23 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U23
Futures_symbol = 'S50U23'
S50U23 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z23
Futures_symbol = 'S50Z23'
S50Z23 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2023 = pd.concat([S50H23, S50M23, S50U23, S50Z23], axis=1)
Options_2023

In [ ]:
# Combined
Normalized_Premium = pd.concat([Options_2013, Options_2014, Options_2015,
                                Options_2016, Options_2017, Options_2018, Options_2019, 
                                Options_2020, Options_2021, Options_2022, Options_2023], 
                               axis=1)
Normalized_Premium